In [1]:
import xml.etree.ElementTree as ET
from cvkg.flextglutils import *

In [2]:
tree = ET.parse('c:/tmp/vk.xml')
registry = tree.getroot()

tags_node = registry.find('tags')
types_node = registry.find('types')
enum_nodes = registry.findall('enums')
feature_nodes = registry.findall('feature')
commands_node = registry.find('commands')

In [3]:
types_node.attrib

{'comment': 'Vulkan type definitions'}

In [4]:
VkGeometryTrianglesNV = types_node[1000]

In [5]:
def node_text(node: ET.Element):
    if node.text is not None:
        return node.text
    return ''.join([node_text(c) for c in node])

In [64]:
from cvkg import SpecParser

parser = SpecParser("c:/tmp/vk.v1.2.203.xml")

Starting to parse 'c:/tmp/vk.v1.2.203.xml':
1508 Types found
510 commands found
215 Enums found
===== Summary =====
Types:     1508
Commands:  510
Enums:     215

=== Parsed 'c:/tmp/vk.v1.2.203.xml' in  121.0 ms ===


In [65]:
print(parser.types['VkResult'])
print(parser.types['VkFramebufferCreateFlagBits'])
print(parser.types['VkImageFormatListCreateInfo'])
print(parser.types['VkColorSpaceKHR'])
print(parser.types['VkDriverIdKHR'])
print(parser.types['VkSamplerCreateFlags'])


[enum] VkResult
[enum] VkFramebufferCreateFlagBits
[struct] VkImageFormatListCreateInfo
  -  sType: VkStructureType
  - ?pNext: const void*
  - ?viewFormatCount: uint32_t
  -  pViewFormats: const VkFormat*
[enum] VkColorSpaceKHR
[enum] VkDriverIdKHR --> VkDriverId
[bitmask] VkSamplerCreateFlags --> VkFlags


In [66]:
print(parser.commands['vkCreateInstance'])
print(parser.commands['vkCmdDispatch'])
print(parser.commands['vkCmdSetDiscardRectangleEXT'])
print(parser.commands['vkGetImageMemoryRequirements2'])

[command] vkCreateInstance(...) -> VkResult
  -  pCreateInfo: const VkInstanceCreateInfo*
  - ?pAllocator: const VkAllocationCallbacks*
  -  pInstance: VkInstance
[command] vkCmdDispatch(...) -> void
  -  commandBuffer: VkCommandBuffer
  -  groupCountX: uint32_t
  -  groupCountY: uint32_t
  -  groupCountZ: uint32_t
[command] vkCmdSetDiscardRectangleEXT(...) -> void
  -  commandBuffer: VkCommandBuffer
  -  firstDiscardRectangle: uint32_t
  -  discardRectangleCount: uint32_t
  -  pDiscardRectangles: const VkRect2D*
[command] vkGetImageMemoryRequirements2(...) -> void
  -  device: VkDevice
  -  pInfo: const VkImageMemoryRequirementsInfo2*
  -  pMemoryRequirements: VkMemoryRequirements2


In [67]:
print(parser.commands['vkCreateImage'])
print(parser.types['VkImageCreateInfo'])

[command] vkCreateImage(...) -> VkResult
  -  device: VkDevice
  -  pCreateInfo: const VkImageCreateInfo*
  - ?pAllocator: const VkAllocationCallbacks*
  -  pImage: VkImage
[struct] VkImageCreateInfo
  -  sType: VkStructureType
  - ?pNext: const void*
  - ?flags: VkImageCreateFlags
  -  imageType: VkImageType
  -  format: VkFormat
  -  extent: VkExtent3D
  -  mipLevels: uint32_t
  -  arrayLayers: uint32_t
  -  samples: VkSampleCountFlagBits
  -  tiling: VkImageTiling
  -  usage: VkImageUsageFlags
  -  sharingMode: VkSharingMode
  - ?queueFamilyIndexCount: uint32_t
  -  pQueueFamilyIndices: const uint32_t*
  -  initialLayout: VkImageLayout


# Consistency check of the parsed data

In [68]:
from cvkg.Types import *

In [69]:
# list enums that are missing a definition of their values
# alias enums are expected to not have values.
undefined_enums = [t for t in parser.types.values() if t.category == 'enum' and not t.alias and t.name not in parser.enums.keys()]
display(undefined_enums)
def unresolved_types(paramsOrMembers: List[Tuple[Parameter,Member]])->List[str]:
    unresolved = []
    for p in paramsOrMembers:
        if p.type not in parser.types:
            unresolved.append(p.type)
    return unresolved
# list types with undefined member types
types_with_undefined_members = [t for t in parser.types.values() if len(unresolved_types(t.members)) > 0]
display(types_with_undefined_members)

[[enum] VkQueryPoolCreateFlagBits,
 [enum] VkInstanceCreateFlagBits,
 [enum] VkDeviceCreateFlagBits]

[]

In [70]:
# list types with undefined member types

def unresolved_types(paramsOrMembers: List[Tuple[Parameter,Member]])->List[str]:
    unresolved = []
    for p in paramsOrMembers:
        if p.type not in parser.types:
            unresolved.append(p.type)
    return unresolved

types_with_undefined_members = [t for t in parser.types.values() if len(unresolved_types(t.members)) > 0]
display(types_with_undefined_members)

[]

In [71]:
# list commands with undefined parameter types
commands_with_undefined_parameters = [c for c in parser.commands.values() if not c.alias and len(unresolved_types(c.params)) > 0]
display(commands_with_undefined_parameters)

[]

In [72]:
# do all `vkCmd` commands have a commandBuffer as a first argument?
vkCmdCommands = [c for c in parser.commands.values() if c.name.startswith('vkCmd') and not c.alias]
print(f"{len(vkCmdCommands)} vkCmd commands")
[c for c in vkCmdCommands if len(c.params) == 0 or c.params[0].type != 'VkCommandBuffer']

144 vkCmd commands


[]

119

In [78]:
[c.name for c in parser.commands.values() if not c.alias and c.name.startswith == 'VkVideo']

[]